In [16]:
import scdrs
import scanpy as sc
sc.set_figure_params(dpi=125)
from anndata import AnnData
from scipy import stats
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
import warnings

warnings.filterwarnings("ignore")

In [17]:
# scdrs.util.load_h5ad is a simple wrapper to read h5ad files
# with basic data filtering
adata = sc.read_h5ad("E:/anaconda/jupyter notebook/bd.h5ad")


# load geneset, convert homologs and overlap gene names to adata.var_names
dict_gs = scdrs.util.load_gs(
    "E:/anaconda/jupyter notebook/geneset.gs",
    src_species="human",
    dst_species="human",
    to_intersect=adata.var_names,
)

# load covariates
#df_cov = pd.read_csv("data/cov.tsv", sep="\t", index_col=0)

# preprocess data to
# (1) regress out from covariates
# (2) group genes into bins by mean and variance
scdrs.preprocess(adata, n_mean_bin=20, n_var_bin=20, copy=False)

In [18]:
dict_df_score = dict()
for trait in dict_gs:
    gene_list, gene_weights = dict_gs[trait]
    dict_df_score[trait] = scdrs.score_cell(
        data=adata,
        gene_list=gene_list,
        gene_weight=gene_weights,
        ctrl_match_key="mean_var",
        n_ctrl=1000,
        weight_opt="vs",
        return_ctrl_raw_score=False,
        return_ctrl_norm_score=True,
        verbose=False,
    )

Computing control scores: 100%|███████████████████████████████████████████████████| 1000/1000 [00:07<00:00, 136.94it/s]


In [4]:
GWASgene=scdrs.method.downstream_gene_analysis(adata, dict_df_score["P"])

In [9]:
TWASgene=scdrs.method.downstream_gene_analysis(adata, dict_df_score["BD_TWAS"])

In [14]:
ATACgene=scdrs.method.downstream_gene_analysis(adata, dict_df_score["BD_ATAC"])

In [20]:
RNAseqgene=scdrs.method.downstream_gene_analysis(adata, dict_df_score["BD_RNAseq"])

In [21]:
RNAseqgene.to_csv("RNAseqgenenew.csv")

In [19]:
atacgene.to_csv("atacgene.csv")
rnaseqgene.to_csv("rnaseqgene.csv")